In [58]:
### Script to compare results of weighted and unweighted MOFA version

#############################################
# Prerequisites - Load Libraries

In [59]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/stark_stemi_R_Env_4_1//lib/R/library"


In [60]:
source('MS4_Plot_Config.r')

###############################################
# Preqrequisites Configurations & Parameters

In [61]:
data_path = '../data/current'

In [62]:
result_path ='../results/current'

In [63]:
data_path

[1] "../data/current"

In [64]:
name = 'V_FINAL_INTEGRATED'  # name of dataset to load

# Load Data 

## MOFA Models

In [65]:
model_names = c('MOFA_MODELV_FINAL_INTEGRATED-FALSE', 'MOFA_MODELV_FINAL_INTEGRATED-TRUE') # define weighted and unweighted model names

In [66]:
model_list = list()

In [67]:
for(i in model_names){
    model_name = paste0(i, '.hdf5') # added number of factors in review to test different 
    outfile = file.path( paste0(result_path, '/E-Analysis/',  model_name) )
    model_list[[as.character(i)]] <- load_model(outfile, verbose = TRUE)
    }

Loading data...

Loading expectations for 2 nodes...

Loading model options...

Loading training options and statistics...

Assigning names to the different dimensions...

Re-ordering factors by their variance explained...

Doing quality control...

Checking views names...

Checking groups names...

Checking samples names...

Checking features names...

Checking dimensions...

Checking there are no features with complete missing values...

Checking sample covariates...

Checking expectations...

Checking for intercept factors...

Warning message in .quality_control(object, verbose = verbose):
“Factor(s) 9, 12, 17 are strongly correlated with the total number of expressed features for at least one of your omics. Such factors appear when there are differences in the total 'levels' between your samples, *sometimes* because of poor normalisation in the preprocessing steps.
”
Checking for highly correlated factors...

Loading data...

Loading expectations for 2 nodes...

Loading model optio

In [68]:
#str(model_list)

## Prepared combined data

In [69]:
path = paste0(result_path, '/E-Analysis/Combined_Data_',name,'.csv')
data_long = read.csv(path)
print(file.info(path)$mtime)

[1] "2024-01-03 14:26:41 CET"


In [70]:
head(data_long,2)

,X,sample_id,variable,value,type
,<int>,<chr>,<chr>,<dbl>,<chr>
1,1,k1,CK,-2.397022,clinical_data
2,2,k10,CK,NA,clinical_data


In [71]:
#data_long[data_long$sample_id == 'm6.1',]

In [72]:
data_long$X = NULL

In [73]:
data_distribution = data_long %>% group_by(variable, type) %>% summarise(mean_val = mean(value, na.rm = TRUE), var_val = var(value, na.rm = TRUE))

`summarise()` has grouped output by 'variable'. You can override using the
`.groups` argument.


# Downstream Analysis of generated models

## Compare total explained variance

In [74]:
lapply(model_list, function(x){
    mean(x@cache$variance_explained$r2_total[[1]])
    })
### 20 Factor model used: 35.7461964551804   

$`MOFA_MODELV_FINAL_INTEGRATED-FALSE`
[1] 35.7462

$`MOFA_MODELV_FINAL_INTEGRATED-TRUE`
[1] 35.70859

In [75]:
### compare explained variance per view

In [76]:
var_explained_per_view = lapply(model_list, function(x){
    x@cache[["variance_explained"]]$r2_total
    })

In [77]:
var_explained_per_view_data = data.frame()

for(i in names(var_explained_per_view)){
    data = var_explained_per_view[[i]]$group1
    data = as.data.frame(data)
    data$view = rownames(data)
    data$amount_factors = i
    
    var_explained_per_view_data = rbind(data, var_explained_per_view_data)
    }

In [78]:
head(var_explained_per_view_data,2)

,data,view,amount_factors
,<dbl>,<chr>,<chr>
clinical_data,100.00000,clinical_data,MOFA_MODELV_FINAL_INTEGRATED-TRUE
cytokine,51.86949,cytokine,MOFA_MODELV_FINAL_INTEGRATED-TRUE


In [79]:
### adjust view naming
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X0_TcellCD4', 'CD4+ T cells (Cluster 0)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'clinical_data', 'Clinical')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X1_TcellCD8', 'CD8+ T cells (Cluster 1)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X10_Bcell', 'B cells (Cluster 10)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X11_TcellCD4', 'CD4+ T cells (Cluster 11)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X12_MonocytesCD16_FCGR3A', 'FCGR3A(high) Monocytes (Cluster 12)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X13_Dendritic', 'Dendritic cells (Cluster 13)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X2_TcellCD4', 'CD4+ T cells (Cluster 2)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X3_NK', 'NK cells (Cluster 3)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X4_MonocytesCD14', 'CD14(high) Monocytes (Cluster 4)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X5_TcellCD4', 'CD4+ T cells (Cluster 5)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X6_MonocytesCD14', 'CD14(high) Monocytes (Cluster 6)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X7_MonocytesCD14', 'CD14(high) Monocytes (Cluster 7)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X8_Bcell','B cells (Cluster 8)')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'X9_MonocytesCD16_FCGR3A','FCGR3A(high) Monocytes (Cluster 9)')


var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'clinical','Clinical')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'cytokine','Cytokine')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'neutrophil','Neutrophil')
var_explained_per_view_data$view = str_replace(var_explained_per_view_data$view, 'proteomics','Proteomics')

In [80]:
unique(var_explained_per_view_data$amount_factors)

[1] "MOFA_MODELV_FINAL_INTEGRATED-TRUE"  "MOFA_MODELV_FINAL_INTEGRATED-FALSE"

In [81]:
figure_name = "REV1_Explained_Variance_Comparison_weighted"

In [82]:
# Sizes of the plot
width_par = 8
height_par = 3.5

In [83]:
g = ggplot(var_explained_per_view_data, aes(x=view, y=data, fill=amount_factors)) + 
    geom_bar(position="dodge", stat="identity") + ylab('amount_explained_variance') + plot_config + coord_flip() + ylab('Explained Variance') +
       scale_fill_discrete(labels=c('MOFA Model (not weighted)', 'MOFA Model (weighted)'))


pdf(paste0('figures/Review_Figures/', figure_name, name, '.pdf'), width =width_par, height =height_par)
print(g)
dev.off()

png 
  2

## Compare factor weights of samples

In [84]:
factor_weights = lapply(model_list, function(x){
    get_factors(x, factors = 'all')$group1
    })

In [85]:
factor_weight_data = data.frame()

for(i in names(factor_weights)){
    data = factor_weights[[i]]
    data = as.data.frame(data)
    data$version = i
    data$sample = rownames(data)
    data = melt(data)
    factor_weight_data = rbind(data, factor_weight_data)
    }

Using version, sample as id variables

Using version, sample as id variables



In [86]:
factor_weight_data$version = paste0(factor_weight_data$variable, '_', str_replace(factor_weight_data$version, 'MOFA_MODELV_FINAL_INTEGRATED', ''))

In [87]:
factor_weight_data$version = str_replace(factor_weight_data$version, '-TRUE', 'weighted')
factor_weight_data$version = str_replace(factor_weight_data$version, '-FALSE', 'unweighted')

In [88]:
head(factor_weight_data,2)

,version,sample,variable,value
,<chr>,<chr>,<fct>,<dbl>
1,Factor1_weighted,k1,Factor1,-2.4794987
2,Factor1_weighted,k10,Factor1,-0.6867466


In [89]:
factor_weight_data = factor_weight_data[factor_weight_data$variable %in% c('Factor1', 'Factor2', 'Factor3', 'Factor4', 'Factor5'),]

In [90]:
cor_data = factor_weight_data %>% dcast(sample ~ version, value.var = 'value')

In [91]:
head(cor_data,2)

,sample,Factor1_unweighted,Factor1_weighted,Factor2_unweighted,Factor2_weighted,Factor3_unweighted,Factor3_weighted,Factor4_unweighted,Factor4_weighted,Factor5_unweighted,Factor5_weighted
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,k1,0.4811325,-2.4794987,-0.2731317,0.8378216,1.4988169,0.33267886,-0.4135678,-0.6309334,1.5052706,-0.2883881
2,k10,0.8920603,-0.6867466,-0.7547770,1.0746178,0.3501023,0.05404922,-0.1524478,-0.6387937,0.4074517,-1.7110773


In [92]:
cor_data$sample = NULL
correlation = cor(cor_data)

In [93]:
head(correlation,2)

,Factor1_unweighted,Factor1_weighted,Factor2_unweighted,Factor2_weighted,Factor3_unweighted,Factor3_weighted,Factor4_unweighted,Factor4_weighted,Factor5_unweighted,Factor5_weighted
Factor1_unweighted,1.00000000,-0.02408174,-0.04058919,0.9649882,-0.01808755,-0.03412484,-3.379073e-05,-0.07229311,0.007509274,0.0766684
Factor1_weighted,-0.02408174,1.00000000,0.86077776,-0.1018771,-0.03923257,0.06228109,5.852193e-02,0.05451766,-0.146495613,0.2080855


In [94]:
correlation = correlation[!is.na(str_extract(rownames(correlation), 'unweighted')),]
correlation = correlation[,is.na(str_extract(colnames(correlation), 'unweighted'))]

In [95]:
correlation

,Factor1_weighted,Factor2_weighted,Factor3_weighted,Factor4_weighted,Factor5_weighted
Factor1_unweighted,-0.02408174,0.96498820,-0.03412484,-0.07229311,0.07666840
Factor2_unweighted,0.86077776,-0.02202818,0.17501517,0.08445746,0.03001498
Factor3_unweighted,-0.03923257,-0.06562592,0.80784720,-0.28370183,0.13387094
Factor4_unweighted,0.05852193,-0.05519457,0.24902620,0.75146050,0.19384986
Factor5_unweighted,-0.14649561,0.06361848,-0.06879140,-0.19657109,-0.54838552


In [96]:
figure_name = "REV2_Correlation_Factor_Vaues_weighted_unweighted"

In [97]:
# Sizes of the plot
width_par = 8
height_par = 4

In [98]:
pdf(paste0('figures/Review_Figures/', figure_name, name, '.pdf'), width =width_par, height =height_par)
correlation_plot = corrplot(correlation, method = 'number', title = '',  pch =20,  mar=c(0,0,2,0), number.cex = 0.8, tl.cex = 0.8, cl.cex =0.8, tl.srt = 90)
dev.off()

png 
  2

## Compare feature weights

In [99]:
weights = lapply(model_list, function(x){
    get_weights(x, views = "all", factors = "all")
    })

In [100]:
weight_data = data.frame()

In [101]:
for(j in names(weights)){
    weights_factor_amount = weights[[j]]

    for (i in names(weights_factor_amount)){
        data = data.frame(weights_factor_amount[[i]])
        data$view = i
        data$version = j
        data$feature = rownames(data)
        data = melt(data)
        weight_data = rbind(weight_data,data)
        }
    }

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id variables

Using view, version, feature as id

In [102]:
head(weight_data,2)

,view,version,feature,variable,value
,<chr>,<chr>,<chr>,<fct>,<dbl>
1,clinical_data,MOFA_MODELV_FINAL_INTEGRATED-FALSE,CK,Factor1,-0.13418597
2,clinical_data,MOFA_MODELV_FINAL_INTEGRATED-FALSE,CK_MB,Factor1,-0.06222621


In [103]:
weight_data$version = paste0(weight_data$variable, '_',str_replace(weight_data$version, 'MOFA_MODELV_FINAL_INTEGRATED', ''))
weight_data$view_feature = paste0(weight_data$view, '__', weight_data$feature)

In [104]:
weight_data$version = str_replace(weight_data$version, '-FALSE', 'unweighted')
weight_data$version = str_replace(weight_data$version, '-TRUE', 'weighted')

In [105]:
head(weight_data,2)

,view,version,feature,variable,value,view_feature
,<chr>,<chr>,<chr>,<fct>,<dbl>,<chr>
1,clinical_data,Factor1_unweighted,CK,Factor1,-0.13418597,clinical_data__CK
2,clinical_data,Factor1_unweighted,CK_MB,Factor1,-0.06222621,clinical_data__CK_MB


In [106]:
weight_data = weight_data[weight_data$variable %in% c('Factor1', 'Factor2', 'Factor3', 'Factor4', 'Factor5'),]

In [107]:
cor_data = weight_data %>% dcast(view_feature ~ version, value.var = 'value')

In [108]:
head(cor_data,2)

,view_feature,Factor1_unweighted,Factor1_weighted,Factor2_unweighted,Factor2_weighted,Factor3_unweighted,Factor3_weighted,Factor4_unweighted,Factor4_weighted,Factor5_unweighted,Factor5_weighted
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,clinical_data__CK,-0.13418597,0.5308249,0.4019222,-0.20966567,-0.1010765,-0.1769928,-0.002177566,2.988978e-10,-0.5093667,1.548127e-01
2,clinical_data__CK_MB,-0.06222621,0.4256853,0.2731985,-0.07963309,-0.2000362,-0.3119296,-0.076098327,-1.560269e-10,-0.1439445,3.505454e-09


In [109]:
cor_data$view_feature = NULL
correlation = cor(cor_data)

In [110]:
head(correlation,2)

,Factor1_unweighted,Factor1_weighted,Factor2_unweighted,Factor2_weighted,Factor3_unweighted,Factor3_weighted,Factor4_unweighted,Factor4_weighted,Factor5_unweighted,Factor5_weighted
Factor1_unweighted,1.00000000,0.09489689,-0.01591703,0.9894728,-0.03084776,-0.02476761,0.01094912,0.01486973,-0.006162275,0.1913241
Factor1_weighted,0.09489689,1.00000000,0.94867525,0.1482241,-0.03052462,0.09253431,0.03367686,0.11456338,0.064739519,-0.3821532


In [111]:
correlation = correlation[!is.na(str_extract(rownames(correlation), 'unweighted')),]
correlation = correlation[,is.na(str_extract(colnames(correlation), 'unweighted'))]

In [112]:
figure_name = "REV2_Correlation_Feature_Weights_weighted_unweighted"

In [113]:
# Sizes of the plot
width_par = 8
height_par = 4

In [114]:
pdf(paste0('figures/Review_Figures/', figure_name, name, '.pdf'), width =width_par, height =height_par)
correlation_plot = corrplot(correlation, method = 'number', title = '',  pch =20,  mar=c(0,0,2,0), number.cex = 0.8, tl.cex = 0.8, cl.cex =0.8, tl.srt = 90)
dev.off()

png 
  2